In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import os

from holodeck import plot, detstats, utils
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo

import hasasia.sensitivity as hsen
import hasasia.sim as hsim
import hasasia.skymap as hsky

In [ ]:
SHAPE = None
NREALS = 500
NFREQS = 40
NLOUDEST = 10

SAVEFIG = False
TOL=0.01
MAXBADS=5

RED_GAMMA = None
RED2WHITE = None

NVARS = 21

NPSRS = 40
NSKIES = 100
TARGET = 'hard_time' # EDIT AS NEEDED
TITLE = plot.PARAM_KEYS[TARGET]  # EDIT AS NEEDED

# Sensitivity Curve EVSS vs. Hard Time

In [ ]:
def get_data(
        target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz', dsc_flag=False, gsc_flag=False,    
):
    load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/data_params.npz' 
    load_dets_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/detstats_s{nskies}' 

    if gsc_flag: load_dets_from_file = load_dets_from_file + '_gsc'
    elif dsc_flag: load_dets_from_file = load_dets_from_file + '_dsc'

    if red_gamma is not None and red2white is not None:
        load_dets_from_file = load_dets_from_file+f'_r2w{red2white:.1f}_rg{red_gamma:.1f}'
    else:
        load_dets_from_file = load_dets_from_file+f'_white'
    load_dets_from_file = load_dets_from_file+'.npz'

    if os.path.exists(load_data_from_file) is False:
        err = f"load data file '{load_data_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    if os.path.exists(load_dets_from_file) is False:
        err = f"load dets file '{load_dets_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    file = np.load(load_data_from_file, allow_pickle=True)
    data = file['data']
    params = file['params']
    file.close()
    print(target, "got data")
    file = np.load(load_dets_from_file, allow_pickle=True)
    print(target, "loaded dets")
    print(file.files)
    dsdat = file['dsdat']
    file.close()

    return data, params, dsdat

def get_ratio_arrays(
        target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, debug=False,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata', dsc_flag=False, gsc_flag=False,
    red=True,     
    ):

    
    filename = path+f'/ratio_arrays_{target}_v{nvars}_r{nreals}_s{nskies}_shape{str(shape)}'

    if gsc_flag: filename = filename + '_gsc'
    elif dsc_flag: filename = filename + '_dsc'
    filename = filename +'.npz'
    
    file = np.load(filename)
    if debug: print(f"{filename}\n{file.files}")
    xx = file['xx_params']
    y0p0 = file['yy_ratio']
    if red:
        y1p5 = file['y1p5_ratio']
        y3p0 = file['y3p0_ratio']
    file.close()
    if red:
        return xx, y0p0, y1p5, y3p0, #y1p5, y3p0
    else:
        return xx, y0p0, 

In [ ]:
def build_arrays():
    target = 'hard_time'
    gsc_flag=True
    dsc_flag=False
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz' 

    # white noise only
    data, params, dsdat = get_data(target, path=path, gsc_flag=gsc_flag, dsc_flag=dsc_flag)
    xx=[]
    yy=[]
    for pp, par in enumerate(params):
        xx.append(params[pp][target])
        dp_bg = np.repeat(dsdat[pp]['dp_bg'], NSKIES).reshape(NREALS, NSKIES)
        dp_ss = dsdat[pp]['ev_ss']
        yy.append(dp_ss/dp_bg)

    filename = path+f'/figdata/ratio_arrays_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}'
    if gsc_flag: filename = filename + '_gsc'
    elif dsc_flag: filename = filename + '_dsc'
    filename = filename +'.npz'

    np.savez(filename, 
                    xx_params = xx, yy_ratio = yy,)
# build_arrays()

In [ ]:
ylabel = plot.LABEL_DPRATIO
targets = [
    'gsmf_phi0',  'mmb_mamp_log10', 'hard_time',  
    'gsmf_mchar0_log10', 'mmb_scatter_dex', 'hard_gamma_inner']
col0p0 = [
    '#336948', '#9e5c41', '#2d839f',
    '#336948', '#9e5c41', '#2d839f',
]
col3p0 = [
    '#8fcf91', '#fda363', '#38cbff',
    '#8fcf91', '#fda363', '#38cbff',
]
col_gw = [
    'k', 'k', 'k',
    'k', 'k', 'k'
]

ii=2 # hardening time
target='hard_time'
print(targets[ii])

fig, ax = plot.figax(xlabel=plot.PARAM_KEYS[target], ylabel=plot.LABEL_EVSS, xscale='linear')
xx, yy_def, = get_ratio_arrays(targets[ii], red=False, dsc_flag=False)
_, yy_dsc = get_ratio_arrays(targets[ii], red=False, dsc_flag=True)
_, yy_gsc = get_ratio_arrays(targets[ii], red=False, gsc_flag=True)


h1, = ax.plot(xx, np.median(yy_dsc, axis=(1,2)), color='tab:blue', alpha=0.9, ls='--', lw=2)
h2, = ax.plot(xx, np.median(yy_def, axis=(1,2)), color='tab:orange', alpha=0.9)
h3, = ax.plot(xx, np.median(yy_gsc, axis=(1,2)), color='tab:green', alpha=0.9, ls=':', lw=3)

colors=['tab:blue', 'tab:orange', 'tab:green']
for pp in [68,]:
    for ii,yy in enumerate([yy_dsc, yy_def, yy_gsc]):
        med, *conf = np.percentile(yy, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.fill_between(xx, *conf, color= colors[ii], alpha=0.25)


# for pp in [50, 95]:
# # for pp in [50]: 
#     med, *conf = np.percentile(yy_dsc, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#     # ax.plot(xx, med, alpha=0.9, color=col3p0[ii])
#     ax.fill_between(xx, *conf, color='tab:orange', alpha=0.25)

ax.legend(handles=[h1, h2, h3], labels=['DSC, rosado-clbrtd','WN, rosado-clbrtd', 'GSC-clbrtd'], loc='lower right')

# Favg

In [ ]:
def build_freq_arrays(target, gsc_flag=False, dsc_flag=False):
    figpath = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata' 
    xx = [] # param
    yy = [] # frequency means
    yy_log = []
    vv = [] # variance
    vv_log = [] # variance of log frequency

    # white noise only
    data, params, dsdat = get_data(target, gsc_flag=gsc_flag, dsc_flag=dsc_flag)
    freqs = data[0]['fobs_cents']
    freqs = np.repeat(freqs, NREALS*NSKIES*NLOUDEST).reshape(NFREQS, NREALS, NSKIES, NLOUDEST)

    for pp, par in enumerate(params):
        xx.append(params[pp][target])
        dpssi = dsdat[pp]['gamma_ssi']
        mean, var2 = detstats.weighted_mean_variance(freqs, weights=dpssi)
        logmean, logvar2 = detstats.weighted_mean_variance(np.log10(freqs), weights=dpssi)
        
        var = np.sqrt(var2)
        logvar = np.sqrt(logvar2)

        yy.append(mean)
        yy_log.append(logmean)
        vv.append(var)
        vv_log.append(logvar)
    saveloc = figpath+f'/freq_meanvar_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}'
    if gsc_flag: saveloc = saveloc+'_gsc'
    elif dsc_flag: saveloc = saveloc+'_dsc'
    saveloc=saveloc+'.npz'
    np.savez(saveloc, xx = xx, yy = yy, vv = vv, yy_log = yy_log, vv_log=vv_log)
xx = []
yy = []

build_freq_arrays('hard_time', gsc_flag=True)
print('on two')
build_freq_arrays('hard_time', dsc_flag=True)

In [ ]:
def get_freq_arrays(target, gsc_flag=False, dsc_flag=False):
    figpath = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata' 
    saveloc = figpath+f'/freq_meanvar_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}'
    if gsc_flag: saveloc = saveloc+'_gsc'
    elif dsc_flag: saveloc = saveloc+'_dsc'
    saveloc=saveloc+'.npz'

    file = np.load(saveloc)
    xx = file['xx']
    yy = file['yy']
    vv = file['vv']
    yy_log = file['yy_log']
    vv_log = file['vv_log']
    file.close()

    return xx, yy, vv, yy_log, vv_log

In [ ]:
fig, ax = plot.figax(
    xlabel=plot.PARAM_KEYS['hard_time'], ylabel='Frequency [yr$^{-1}$]', xscale='linear')



xx, _, _, yy_log_has, vv_log_has = get_freq_arrays('hard_time', dsc_flag=True)
_, _, _, yy_log_def, vv_log_def = get_freq_arrays('hard_time', dsc_flag=False)
_, _, _, yy_log_gsc, vv_log_gsc = get_freq_arrays('hard_time', gsc_flag=True)
vv_logs = [vv_log_has, vv_log_def, vv_log_gsc]

colors=['tab:blue', 'tab:orange', 'tab:green']
labels=['DSC, rosado-clbrtd','WN, rosado-clbrtd', 'GSC-clbrtd']

handles=[]
for ii, yy_log in enumerate([yy_log_has, yy_log_def, yy_log_gsc]):
    vv_log = vv_logs[ii]
    yy_min = 10**(yy_log - vv_log) *YR
    yy_max = 10**(yy_log + vv_log) *YR
    yy = 10**yy_log * YR
    
    hh, = ax.plot(xx, yy, color=colors[ii], alpha=0.9)
    handles.append(hh)
    ax.fill_between(xx, yy_min, yy_max, color=colors[ii], alpha=0.2, )

ax.legend(handles=handles, labels=labels)

In [ ]:





y1 = dp_ssi_clbrt_has[:,:,:,0].reshape(NFREQS, NREALS*NSKIES)
label1 = 'noise = DeterSC + S_rest'
y2 = dp_ssi_clbrt_def[:,:,:,0].reshape(NFREQS, NREALS*NSKIES)
label2 = 'noise = S_WN + S_rest'
y3 = _dsdat_has_clbrt['gamma_ssi'][:,:,:,0].reshape(NFREQS, NREALS*NSKIES)
label3 = 'hasasia GSC-calibrated'

colors = ['tab:blue', 'tab:orange', 'tab:green']
handles = []

for ii,yy  in enumerate([y1, y2,y3]):
    hh = plot.draw_med_conf_color(ax, xx, yy, color=colors[ii])
    handles.append(hh)


var2s = [var2_clbrt_has, var2_clbrt_def, var2_clbrt_sc]
colors = ['blue', 'orangered', 'limegreen']
for ii,favg in enumerate([favg_clbrt_has, favg_clbrt_def, favg_clbrt_sc]):
    std = np.sqrt(var2s[ii])
    hh = ax.axvline(favg, color=colors[ii], linestyle='--')
    handles.append(hh)
    # ax.axvspan(favg-std, favg+std, alpha=0.2, color=colors[ii])

y4 = _dsdat_has['dp_bg']
y5 = _dsdat_def['dp_bg']
y6 = _dsdat_has_clbrt['dp_bg']
colors = ['darkblue', 'saddlebrown', 'darkgreen']
lws = [3,2,1]
for ii,yy in enumerate([y4, y5, y6]):
    hh = ax.axhline(np.median(yy), color=colors[ii], lw=lws[ii])
    handles.append(hh)
    for pp in [50, 95]:
        percs = (50-pp/2, 50+pp/2)
    ax.axhspan(*np.percentile(yy, percs), color=colors[ii], alpha=0.2)


labels = [label1, label2, label3, 'favg (DSC)', 'favg (WN)', 'favg (GSC-clbrtd)', 'DP_BG (GSC)', 'DP_BG (WN)', 'DP_BG (GSC-clbrtd)']
ax.legend(handles=handles, labels=labels, loc='upper right')
